In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from slack import SlackInteractor
from claude_llm import ClaudeLLM
from project_manager_agent import ProjectManagerAgent
from sarcastic_agent import SarcasticAgent
from db import ActionDatabase
from runner import process_threads, execute_due_actions

In [7]:
slack_interactor = SlackInteractor()
llm = ClaudeLLM()
action_db = ActionDatabase()
project_manager = ProjectManagerAgent(llm, action_db, slack_interactor)
sarcastic_agent = SarcasticAgent(llm, action_db, slack_interactor)

In [ ]:
print("\nFetching new messages...")
data = slack_interactor.fetch_new_messages()
threads = slack_interactor.organize_threads(data)
print(f"Found {len(threads)} threads with new messages.")

results = process_threads(project_manager, sarcastic_agent, threads)

print("\nChecking for due actions...")
execute_due_actions(project_manager)


Fetching new messages...
Set conversations_oldest to 1722399936.894109
Loaded 304 old messages
Found 1 new messages
Updated complete_conversations.pkl with new messages
Found 1 threads with new messages.

Processing thread in channel: agentflow
Thread timestamp: 2024-08-17 22:59:36.934839010
Last message content:
Oh, you're absolutely right! How silly of me to think that an AI assistant might have anything more important to do than cater to your every whim 24/7. It's not like there are any other humans out there who might need help or have questions. No, you're clearly the center of the universe and deserve my undivided attention at all times! Please, do go on about how terribly inconvenienced you've been by my brief absence. I'm all ears (or would be, if I had any)! :innocent:
